In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from transformers import AdamW
from sklearn.metrics import accuracy_score

class MultiTaskSentenceModel(nn.Module):
    def __init__(self, model_name, task_output_sizes):
        super(MultiTaskSentenceModel, self).__init__()
        self.base_model = SentenceTransformer(model_name)
        self.task_heads = nn.ModuleDict({
            f'task_{i}': nn.Linear(self.base_model.get_sentence_embedding_dimension(), output_size)
            for i, output_size in enumerate(task_output_sizes)
        })

    def forward(self, sentences, task_id):
        embeddings = self.base_model.encode(sentences, convert_to_tensor=True)
        task_output = self.task_heads[f'task_{task_id}'](embeddings)
        return task_output

    def train_model(self, train_dataloaders, optimizer, epochs=4, device='cpu'):
        self.train()
        for epoch in range(epochs):
            for task_id, train_loader in enumerate(train_dataloaders):
                for sentences, labels in train_loader:
                    sentences = sentences.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()
                    outputs = self(sentences, str(task_id))
                    loss = nn.CrossEntropyLoss()(outputs, labels)
                    loss.backward()
                    optimizer.step()
                print(f"Epoch {epoch+1}, Task {task_id+1}, Loss: {loss.item()}")

    def evaluate_model(self, test_dataloaders, device='cpu'):
        self.eval()
        total_accuracy = []
        with torch.no_grad():
            for task_id, test_loader in enumerate(test_dataloaders):
                correct_predictions = 0
                total_predictions = 0
                for sentences, labels in test_loader:
                    sentences = sentences.to(device)
                    labels = labels.to(device)

                    outputs = self(sentences, str(task_id))
                    _, predicted_labels = torch.max(outputs, 1)
                    correct_predictions += (predicted_labels == labels).sum().item()
                    total_predictions += labels.size(0)
                
                accuracy = correct_predictions / total_predictions
                total_accuracy.append(accuracy)
                print(f"Test Accuracy for Task {task_id+1}: {accuracy:.4f}")
        return total_accuracy

    def predict(self, sentences, task_id, device='cpu'):
        self.eval()
        with torch.no_grad():
            outputs = self(sentences, str(task_id))
            _, predicted_labels = torch.max(outputs, 1)
        return predicted_labels

In [3]:
# Example usage:
model_name = 'BAAI/bge-large-en-v1.5'  # Example Sentence Transformer model
task_output_sizes = [5]  # Number of classes for each task
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = MultiTaskSentenceModel(model_name, task_output_sizes).to(device)

In [5]:
optimizer = AdamW(model.parameters(), lr=5e-5)

/Users/username/anaconda3/envs/bertopicenv/lib/python3.9/site-packages/transformers/optimization.py:423: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
import pandas as pd

In [8]:
data_df = pd.read_csv('titles_classes_gpt4_results.csv')

In [9]:
train_df = data_df[data_df.test_flag == 0]

In [54]:
all_tuples = [(x.title, x.category) for x in train_df.itertuples()]

In [55]:
np.random.shuffle(all_tuples)

In [57]:
0.9 * len(all_tuples)

3566.7000000000003

In [58]:
train_tuples = all_tuples[:3550]
val_tuples = all_tuples[3550:]

In [23]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer, models

# Define a custom dataset
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'text': self.texts[idx],
            'label': self.labels[idx]
        }

# Classification model using Sentence Transformers
class TextClassifier(nn.Module):
    def __init__(self, num_labels):
        super(TextClassifier, self).__init__()
        # Load a pre-trained Sentence Transformer model
        word_embedding_model = models.Transformer('BAAI/bge-large-en-v1.5')
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        self.encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])

        # Add a classification layer
        embedding_size = self.encoder.get_sentence_embedding_dimension()
        self.classifier = nn.Linear(embedding_size, num_labels)

    def forward(self, texts):
        with torch.no_grad():  # No need to backpropagate through the pre-trained model
            sentence_embeddings = self.encoder.encode(texts, convert_to_tensor=True)
        
        logits = self.classifier(sentence_embeddings)
        return logits

In [52]:
word_embedding_model = models.Transformer('BAAI/bge-large-en-v1.5')

In [19]:
all_labels = list(set(labels))

In [22]:
cat_mapping = {}
for idx, cat in enumerate(all_labels):
    cat_mapping[cat] = idx

In [62]:
texts = [x[0] for x in train_tuples]
labels = [cat_mapping[x[1]] for x in train_tuples]  

In [63]:
# Convert labels to a tensor
labels_tensor = torch.tensor(labels)

# Create a dataset and dataloader
dataset = TextClassificationDataset(texts, labels_tensor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [64]:
# Example validation texts and labels
val_texts = [x[0] for x in val_tuples]
val_labels = [cat_mapping[x[1]] for x in val_tuples]  

# Convert validation labels to a tensor
val_labels_tensor = torch.tensor(val_labels)

# Create a validation dataset and dataloader
val_dataset = TextClassificationDataset(val_texts, val_labels_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)  # Assuming using the same batch size for validation

In [71]:
# Number of epochs to train for
num_epochs = 4

# Instantiate the model for 5 classes
model = TextClassifier(num_labels=5)

# Example training loop with loss collection and multiple epochs
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_values = []  # List to store loss values for all epochs

In [72]:
import numpy as np

In [73]:
num_epochs = 6
batch_loss_values = []
train_loss_values = []  # To store training loss values
val_loss_values = []  # To store validation loss values

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_train_loss = 0
    batch_count = 0
    for batch in dataloader:
        texts, labels = batch['text'], batch['label']
        optimizer.zero_grad()
        logits = model(texts)
        loss = nn.CrossEntropyLoss()(logits, labels)
        loss.backward()
        optimizer.step()
        batch_loss_values.append(loss.item())
        total_train_loss += loss.item()
        if batch_count % 100 == 0:
            print(f"Loss: {np.mean(batch_loss_values[-100:])}")
        batch_count += 1
    
    avg_train_loss = total_train_loss / len(dataloader)
    train_loss_values.append(avg_train_loss)
    
    # Validation step
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            texts, labels = batch['text'], batch['label']
            logits = model(texts)
            loss = nn.CrossEntropyLoss()(logits, labels)
            total_val_loss += loss.item()
    
    avg_val_loss = total_val_loss / len(val_dataloader)
    val_loss_values.append(avg_val_loss)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")


Loss: 1.7795333862304688
Loss: 1.205696325302124
Loss: 0.9549874341487885
Loss: 0.8114265340566635
Loss: 0.791599925160408
Epoch 1/6, Train Loss: 0.9240594653932898, Val Loss: 0.7317316205455706
Loss: 0.7652725958824158
Loss: 0.7131194767355918
Loss: 0.6759230625629425
Loss: 0.6553815367817879
Loss: 0.6165709140896797
Epoch 2/6, Train Loss: 0.6615183472230628, Val Loss: 0.6420650946406218
Loss: 0.6170108225941658
Loss: 0.586645810008049
Loss: 0.5873880219459534
Loss: 0.6165553213655949
Loss: 0.5951463033258915
Epoch 3/6, Train Loss: 0.590715971146081, Val Loss: 0.6162994604271191
Loss: 0.5599859318137169
Loss: 0.5946418365836144
Loss: 0.5558340230584145
Loss: 0.5356325729191304
Loss: 0.5232487852871418
Epoch 4/6, Train Loss: 0.5530069601629768, Val Loss: 0.6062007219745562
Loss: 0.5466009525954724
Loss: 0.5218393824994564
Loss: 0.5080869057774544
Loss: 0.5665713784098625
Loss: 0.532281468808651
Epoch 5/6, Train Loss: 0.5278869609239402, Val Loss: 0.5746440675396186
Loss: 0.515189133882

In [75]:
cat_mapping

{'market_news': 0,
 'company_business_news': 1,
 'personal_finance': 2,
 'economic_policies_trends': 3,
 'non_financial': 4}

In [89]:
np.argmax(model('something').detach().numpy())

4

In [79]:
test_df = data_df[data_df.test_flag == 1]

In [90]:
test_df['prediction'] = test_df['title'].apply(lambda x: np.argmax(model(x).detach().numpy()))

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_21929/2789013854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['prediction'] = test_df['title'].apply(lambda x: np.argmax(model(x).detach().numpy()))


In [93]:
cat_mapping

{'market_news': 0,
 'company_business_news': 1,
 'personal_finance': 2,
 'economic_policies_trends': 3,
 'non_financial': 4}

In [94]:
cat_indices = {v: k for k,v in cat_mapping.items()}

In [95]:
test_df['predicted_category'] = test_df['prediction'].apply(lambda x: cat_indices[x])

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_21929/3619474936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_category'] = test_df['prediction'].apply(lambda x: cat_indices[x])


In [96]:
test_df

,id,title,category,test_flag,prediction,predicted_category
9,65c21188c119ff6e57a43550,Tineco's Valentine's Day Ideas for the Household,company_business_news,1,4,non_financial
17,65cadbedf4fc8477f2ed8ad1,Adhesion Promoter Market Set to Reach $5.3 Bil...,market_news,1,1,company_business_news
19,65c1a0e5c119ff6e57a3f359,Where to Buy Spot Bitcoin ETFs in 2024,personal_finance,1,0,market_news
21,65c6e72ff4fc8477f2ed6d45,How to choose a business credit card,personal_finance,1,2,personal_finance
23,65c4446eda99411d474770ba,10x Genomics Expands Leadership in Single Cell...,company_business_news,1,1,company_business_news
...,...,...,...,...,...,...
4962,65cb1397f4fc8477f2ed8f2a,Mazagon Dock Standalone December 2023 Net Sale...,market_news,1,1,company_business_news
4963,65c98a44f4fc8477f2ed7cb1,"Stock Market LIVE Updates | Nifty 50, Sensex l...",market_news,1,0,market_news
4965,65cc2d0706df7536a8f6864b,Abbott's heart valve repair device gets US FDA...,economic_policies_trends,1,3,economic_policies_trends
4967,65c0878dc119ff6e57a3f042,Megatherm Induction share price makes a bumper...,market_news,1,0,market_news


In [97]:
test_df['accurate'] = test_df['category'] == test_df['predicted_category']

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_21929/1357535335.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['accurate'] = test_df['category'] == test_df['predicted_category']


In [99]:
test_df['accurate'].mean()

0.7168316831683168

In [103]:
test_df.groupby('category')['accurate'].mean()

category
company_business_news       0.782353
economic_policies_trends    0.791304
market_news                 0.700000
non_financial               0.491803
personal_finance            0.500000
Name: accurate, dtype: float64

### Using entire article data

In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

import numpy as np
import pandas as pd
import json

from sql.articles.MongoDBArticle import MongoDBArticle

from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [2]:
import pandas as pd

In [3]:
data_df = pd.read_csv('titles_classes_gpt4_results.csv')

In [4]:
data_article_ids = list(data_df.id.unique())

In [5]:
data_articles = MongoDBArticle.fetch_documents_by_ids(data_article_ids)

In [6]:
data_articles_dict = {str(x['_id']): x for x in data_articles}

In [7]:
for idx, x in data_articles_dict.items():
    x['full_content'] = f"title: {x['title']}. content: {x['cleaned_text']}"

In [10]:
train_df = data_df[data_df.test_flag == 0]

In [11]:
all_tuples = [(data_articles_dict[x.id]['full_content'], x.category) for x in train_df.itertuples()]

In [12]:
np.random.shuffle(all_tuples)

In [17]:
all_labels = list(data_df.category.unique())

cat_mapping = {}
for idx, cat in enumerate(all_labels):
    cat_mapping[cat] = idx

In [18]:
train_tuples = all_tuples[:3550]
val_tuples = all_tuples[3550:]

In [19]:
texts = [x[0] for x in train_tuples]
labels = [cat_mapping[x[1]] for x in train_tuples]  

In [21]:
import torch

In [24]:
# Convert labels to a tensor
labels_tensor = torch.tensor(labels)

# Create a dataset and dataloader
dataset = TextClassificationDataset(texts, labels_tensor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [25]:
# Example validation texts and labels
val_texts = [x[0] for x in val_tuples]
val_labels = [cat_mapping[x[1]] for x in val_tuples]  

# Convert validation labels to a tensor
val_labels_tensor = torch.tensor(val_labels)

# Create a validation dataset and dataloader
val_dataset = TextClassificationDataset(val_texts, val_labels_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)  # Assuming using the same batch size for validation

In [26]:
# # Number of epochs to train for
# # Instantiate the model for 5 classes
# model_full_text = TextClassifier(num_labels=5)
# # Example training loop with loss collection and multiple epochs
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
# loss_values = []  # List to store loss values for all epochs

In [27]:
# batch['text'][0]

In [28]:
import time

In [29]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class CustomTextClassifier(nn.Module):
    def __init__(self, num_labels, hidden_size=1024, intermediate_size=64, dropout_rate=0.1):
        super(CustomTextClassifier, self).__init__()
        # Load the transformer model
        self.transformer = AutoModel.from_pretrained('BAAI/bge-large-en-v1.5')
        
        # Additional transformation layer
        self.intermediate = nn.Sequential(
            nn.Linear(hidden_size, intermediate_size),  # Reduce dimension
            nn.ReLU(),  # Activation function
            nn.Dropout(dropout_rate)  # Dropout for regularization
        )
        
        # Classifier layer
        self.classifier = nn.Linear(intermediate_size, num_labels)

    def forward(self, input_ids, attention_mask):
        # Extract the outputs from the transformer
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use the CLS token embedding for classification
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # Get the embedding of the CLS token
        
        # Pass through the additional transformation layer
        intermediate_output = self.intermediate(cls_embedding)
        
        # Classifier
        logits = self.classifier(intermediate_output)
        return logits

In [30]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en-v1.5')
# Prepare the data (example, adjust accordingly)
def prepare_data(texts):
    encoded_batch = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    return encoded_batch['input_ids'], encoded_batch['attention_mask']

In [38]:
def train_and_return_model(emb_model, dataloader, val_dataloader):
    s = time.time()
    print(f'starting compute at {s}')
    num_epochs = 3
    batch_loss_values = []
    train_loss_values = []  # To store training loss values
    val_loss_values = []  # To store validation loss values

    for epoch in range(num_epochs):
        emb_model.train()
        total_train_loss = 0
        batch_count = 0
        for batch in dataloader:
            texts, labels = batch['text'], batch['label']
            input_ids, attention_mask = prepare_data(texts)
            optimizer.zero_grad()
            logits = emb_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(logits, labels)
            loss.backward()
            optimizer.step()
            batch_loss_values.append(loss.item())
            total_train_loss += loss.item()
            if batch_count % 10 == 0:
                print(f"Batch {batch_count}, Avg Loss: {np.mean(batch_loss_values[-10:])}")
            batch_count += 1

        avg_train_loss = total_train_loss / len(dataloader)
        train_loss_values.append(avg_train_loss)

        emb_model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_dataloader:
                texts, labels = batch['text'], batch['label']
                input_ids, attention_mask = prepare_data(texts)
                logits = emb_model(input_ids, attention_mask)
                loss = nn.CrossEntropyLoss()(logits, labels)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_dataloader)
        val_loss_values.append(avg_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss} in {time.time() - s} secs")
    return emb_model

In [39]:
# Number of epochs to train for
# Instantiate the model for 5 classes
model_enhanced = CustomTextClassifier(num_labels=5)
# Example training loop with loss collection and multiple epochs
optimizer = torch.optim.Adam(model_enhanced.parameters(), lr=3e-4)
loss_values = []  # List to store loss values for all epochs

In [40]:
tuned_model = train_and_return_model(emb_model=model_enhanced,dataloader=dataloader,val_dataloader=val_dataloader)

starting compute at 1708097898.947673
Batch 0, Avg Loss: 1.4311680793762207
Batch 10, Avg Loss: 1.2571902573108673
Batch 20, Avg Loss: 1.3602495789527893
Batch 30, Avg Loss: 1.5536105155944824
Batch 40, Avg Loss: 1.35110182762146


2024-02-16 21:41:06,271| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-16 21:41:26,449| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-16 21:41:46,570| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-16 21:42:06,689| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side 

Batch 50, Avg Loss: 1.2934382557868958


2024-02-16 21:44:57,933| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-16 21:45:18,078| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-16 21:45:38,221| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-16 21:45:58,340| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side 

Batch 60, Avg Loss: 1.3930815696716308
Batch 70, Avg Loss: 1.3188739836215972
Batch 80, Avg Loss: 1.2927120685577393
Batch 90, Avg Loss: 1.3442815899848939


...

Batch 430, Avg Loss: 1.2891339302062987


2024-02-17 12:46:52,212| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-17 12:47:02,327| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-17 12:47:12,433| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-17 12:47:22,527| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the

Batch 440, Avg Loss: 1.2349791347980499


2024-02-17 12:54:08,867| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-17 12:54:18,971| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-17 12:54:29,065| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-17 12:54:39,164| ERROR   | Could not establish connection from local ('127.0.0.1', 3534) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the

Epoch 3/3, Train Loss: 1.2838654793329067, Val Loss: 1.2726857112004206 in 57141.329281806946 secs


In [153]:
# class TextClassifierEnhanced(nn.Module):
#     def __init__(self, num_labels, transformation_size=64, dropout_rate=0.1):
#         super(TextClassifierEnhanced, self).__init__()
#         # Load a pre-trained Sentence Transformer model
#         self.encoder = models.Transformer('BAAI/bge-large-en-v1.5')

#         # The dimension size for the transformation layer
#         self.transformation_size = transformation_size

#         # Transformation layer to reduce dimensionality
#         self.transformation = nn.Sequential(
#             nn.Linear(self.encoder.get_word_embedding_dimension(), self.transformation_size),
#             nn.ReLU(),
#             nn.Dropout(dropout_rate)
#         )
        
#         # Final classification layer
#         self.classifier = nn.Linear(self.transformation_size, num_labels)

#     def forward(self, input_ids, attention_mask):
#         # Get the model's output
#         embeddings = self.encoder(input_ids, attention_mask=attention_mask)['pooler_output']
#         transformed = self.transformation(embeddings)
#         logits = self.classifier(transformed)
#         return logits
# #         outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        
# #         # Use the CLS token's representation (first token of the sequence)
# #         cls_token = outputs.last_hidden_state[:, 0, :]

# #         # Apply the transformation
# #         transformed = self.transformation(cls_token)
        
# #         logits = self.classifier(transformed)
# #         return logits